In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
import traceback
import requests
import json

In [2]:
def get_all_urls(base_url, max_urls=30):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    
    # Automatically manage the ChromeDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    driver.get(base_url)
    
    all_urls = set()

    while len(all_urls) < max_urls:
        try:
            # Scroll the element into view
            load_more_button = driver.find_element(By.ID, "loadmoretxt")
            driver.execute_script("arguments[0].scrollIntoView(true);", load_more_button)
            time.sleep(1)  # Give it a moment to scroll
            
            # Click the "Load More" button
            load_more_button.click()
            time.sleep(2)  # Adjust this delay as needed

            # Parse the page after each click
            soup = BeautifulSoup(driver.page_source, "html.parser")
            links = soup.select("#moreistiarea a")
            
            # Add URLs to the set (to avoid duplicates)
            for link in links:
                href = link.get("href")
                if href:
                    full_url = urljoin(base_url, href)
                    all_urls.add(full_url)

            # Stop if we have collected enough URLs
            if len(all_urls) >= max_urls:
                break

        except Exception as e:
            print("Exception occurred while clicking 'Load More':", e)
            traceback.print_exc()  # Print the full stack trace for debugging
            break

    # Close the browser
    driver.quit()

    # Limit the result to `max_urls`
    return list(all_urls)[:max_urls]

# Example usage
base_url = "https://www.islamweb.net/ar/fatwa/"
all_urls = get_all_urls(base_url)

for url in all_urls:
    print(url)


https://www.islamweb.net/ar/fatwa/497086/نذر-صلاة-ركعتين-بعد-كل-فريضة-إن-قضيت-حاجته-ويريد-التحلل-منه
https://www.islamweb.net/ar/fatwa/497191/كفارة-الحنث-في-اليمين-عدة-مرات
https://www.islamweb.net/ar/fatwa/496969/حكم-إرسال-نفقة-الزوجة-عن-طريق-أم-الزوج-مع-إمكان-إرسالها-لها-مباشرة
https://www.islamweb.net/ar/fatwa/498040/حكم-بيع-ثمار-الأشجار-المتنوعة-عند-بدو-صلاح-بعضها
https://www.islamweb.net/ar/fatwa/497554/تناول-المكملات-الغذائية-بطيئة-الامتصاص-قبل-طلوع-الفجر-في-رمضان
https://www.islamweb.net/ar/fatwa/497383/حكم-الاعتمار-على-نفقة-شخص-ماله-حرام-أو-مختلط
https://www.islamweb.net/ar/fatwa/497332/أَخْذُ-أَحَدِ-الشريكين-شيئًا-من-المال-قبل-تقسيم-الأرباح
https://www.islamweb.net/ar/fatwa/496951/إلزام-الشخص-نفسه-بأذكار-محددة-يوميا
https://www.islamweb.net/ar/fatwa/497158/إنفاق-المال-في-سداد-دين-عن-ميت-أم-إبقاؤه-للاحتياجات-المستقبلية
https://www.islamweb.net/ar/fatwa/497422/مات-عن-زوجة-وثلاثة-إخوة-أشقاء-وأخت-شقيقة-وأختين-من-الأم
https://www.islamweb.net/ar/fatwa/497788/أحكام-الرجوع-في-الهبة-ل

In [5]:
def scrape_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Ensure correct encoding is used (UTF-8)
        response.encoding = "utf-8"

        soup = BeautifulSoup(response.text, "html.parser")

        # Extract all <p> tags from the page
        paragraphs = soup.find_all("p")

        # Clean and store the paragraphs
        clean_paragraphs = [p.get_text(strip=True) for p in paragraphs]

        # Join the clean paragraphs into a single text
        combined_text = " ".join(clean_paragraphs)

        return {"url": url, "combined_text": combined_text}

    except requests.RequestException as e:
        print(f"Error scraping {url}: {e}")
        return None


url = all_urls[0]
result = scrape_page(url)

print(result["combined_text"])

قبل: 10 سنين نذرت أن أصلي ركعتين بعد كل صلاة إن قضيت حاجتي، وزدتها بركعتين أيضا بعد صلاة المغرب، لتصبح: 4 ركعات، وبالفعل بدأت أصلي إلى يومنا هذا، لكنني أتكاسل عنها إن فاتني فرض، حيث أنظر إلى أنني سأصلي فرضين مع النذر، بمجموع: 11 ركعة، فأترك الصلاة، ولا أخشع أصلا، وأعلم أنه يجب علي أن أخشع، وأن لا أتكاسل، وأعلم أنني لن أفعل -وكما أسلفت- فإنني أترك صلاة الفرض، وأعود لها تائبا بسبب نذري الذي سبب لي ترك الفروض، فهل يحق لي ترك هذا النذر، لتجنب الترك المتكرر للفرض بسبب الكسل؟ الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعد:فتلك الصلاة المنذورة صارت لازمة لك كالفريضة، ولا سعة لك في تركها، بل لا بد من الحفاظ عليها، كما تحافظ على الفرائض، لأنك الذي ألزمت نفسك بها، وقد قال النبي صلى الله عليه وسلم:من نذر أن يطيع الله فليطعه.ولا حيلة في التخلص من ذلك النذر ما دمت قادرا على الوفاء به، وليس الكسل عذرا يبيح ترك الوفاء به، فعليك أن تحافظ على الفرائض، وتلك المنذورات، ولا تخل بها لما ذكرنا، وانظر للفائدة الفتويين:136409،380109.والله أعلم. الحمد لله والصلاة والسلام على رسول الله وعلى آله

In [14]:
def scrape_fatwa_islamweb(base_url, max_urls=30):
    print(f"Starting to scrape with base URL: {base_url}")
    print(f"Maximum URLs to scrape: {max_urls}")
    
    all_urls = get_all_urls(base_url, max_urls=max_urls)
    print(f"Retrieved {len(all_urls)} URLs to scrape")
    
    scraped_data = []

    for index, url in enumerate(all_urls, 1):
        print(f"Scraping URL {index}/{len(all_urls)}: {url}")
        result = scrape_page(url)
        if result:
            scraped_data.append(result)
            print(f"Successfully scraped data from URL {index}")
        else:
            print(f"Failed to scrape data from URL {index}")
        
        if index % 10 == 0:
            print(f"Progress: {index}/{len(all_urls)} URLs processed")

    print(f"Scraping complete. Scraped {len(scraped_data)} URLs successfully out of {len(all_urls)} total URLs")
    return scraped_data

In [20]:
scraped_data = scrape_fatwa_islamweb("https://www.islamweb.net/ar/fatwa/",170)

Starting to scrape with base URL: https://www.islamweb.net/ar/fatwa/
Maximum URLs to scrape: 170
Retrieved 170 URLs to scrape
Scraping URL 1/170: https://www.islamweb.net/ar/fatwa/495583/حكم-الحصول-على-مال-من-دولاب-الحظ
Successfully scraped data from URL 1
Scraping URL 2/170: https://www.islamweb.net/ar/fatwa/495415/لا-يشترط-دخول-الزوج-الثاني-لصحة-رجعة-الزوج-الأول-الذي-طلَّق-دون-الثلاث
Successfully scraped data from URL 2
Scraping URL 3/170: https://www.islamweb.net/ar/fatwa/496834/أوصت-بأن-تأخذ-أخواتها-ما-يردن-من-ملابسها-وحقائبها-وتوزيع-الباقي-على-المحتاجين
Successfully scraped data from URL 3
Scraping URL 4/170: https://www.islamweb.net/ar/fatwa/497191/كفارة-الحنث-في-اليمين-عدة-مرات
Successfully scraped data from URL 4
Scraping URL 5/170: https://www.islamweb.net/ar/fatwa/495433/حكم-صلاة-من-أتى-فيها-بذكر-زائد-في-غير-موضعه
Successfully scraped data from URL 5
Scraping URL 6/170: https://www.islamweb.net/ar/fatwa/494944/الاعتبارات-الشرعية-في-تحريم-تأجير-الشهادة-الجامعية
Successfully sc

In [21]:
with open('scraped_fatwas.json', 'w', encoding='utf-8') as json_file:
    json.dump(scraped_data, json_file, ensure_ascii=False, indent=4)

print("Scraped data saved to 'scraped_fatwas.json'")

Scraped data saved to 'scraped_fatwas.json'
